# TI4 Map Balance Analysis

This notebook demonstrates the scientific analysis and visualization capabilities of the TI4 analysis toolkit.

We'll cover:
1. Creating a sample TI4 map
2. Analyzing balance using the iterative optimization algorithm
3. Computing advanced spatial statistics (Moran's I, Getis-Ord Gi*)
4. Visualizing results with publication-quality plots

In [1]:
# Imports
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from ti4_analysis.algorithms.hex_grid import HexCoord, hex_distance, get_hexes_in_range
from ti4_analysis.data.map_structures import (
    Planet, System, MapSpace, MapSpaceType, Evaluator, 
    PlanetTrait, TechSpecialty, PlanetEvalStrategy
)
from ti4_analysis.algorithms.balance_engine import (
    TI4Map, improve_balance, analyze_balance, get_home_values, get_balance_gap
)
from ti4_analysis.visualization.map_viz import (
    plot_hex_map, plot_balance_convergence, plot_balance_distribution,
    create_balance_report, plot_value_heatmap
)
from ti4_analysis.spatial_stats.spatial_metrics import (
    comprehensive_spatial_analysis, morans_i, create_adjacency_weights,
    jains_fairness_index, identify_hotspots
)

# Set style
sns.set_style('whitegrid')
plt.rcParams['figure.dpi'] = 100

print("✓ Imports successful")

ModuleNotFoundError: No module named 'ti4_analysis'

## 1. Create Sample Map

Let's create a simple 6-player map to demonstrate the analysis.

In [ ]:
def create_sample_map():
    """Create a sample 6-player TI4 map."""
    spaces = []
    
    # Create home systems in a ring
    home_coords = [
        HexCoord(0, -3, 3),
        HexCoord(3, -3, 0),
        HexCoord(3, 0, -3),
        HexCoord(0, 3, -3),
        HexCoord(-3, 3, 0),
        HexCoord(-3, 0, 3)
    ]
    
    for coord in home_coords:
        spaces.append(MapSpace(coord, MapSpaceType.HOME))
    
    # Add Mecatol Rex at center
    mecatol = Planet("Mecatol Rex", resources=1, influence=6)
    mecatol_system = System(id=18, planets=[mecatol])
    spaces.append(MapSpace(HexCoord(0, 0, 0), MapSpaceType.SYSTEM, mecatol_system))
    
    # Create various planet systems
    sample_systems = [
        # Rich systems
        (HexCoord(1, -2, 1), Planet("Tar'Mann", 3, 1, [PlanetTrait.INDUSTRIAL], [TechSpecialty.BIOTIC])),
        (HexCoord(2, -2, 0), Planet("Wellon", 1, 2, [PlanetTrait.INDUSTRIAL], [TechSpecialty.CYBERNETIC])),
        (HexCoord(-1, -2, 3), Planet("Arinam", 1, 2, [PlanetTrait.INDUSTRIAL])),
        
        # Medium systems
        (HexCoord(2, -1, -1), Planet("Mehar Xull", 1, 3, [PlanetTrait.HAZARDOUS], [TechSpecialty.WARFARE])),
        (HexCoord(1, 1, -2), Planet("Quann", 2, 1, [PlanetTrait.CULTURAL])),
        (HexCoord(-1, 2, -1), Planet("Lodor", 3, 1, [PlanetTrait.CULTURAL])),
        
        # Weaker systems
        (HexCoord(-2, 1, 1), Planet("New Albion", 1, 1, [PlanetTrait.INDUSTRIAL], [TechSpecialty.BIOTIC])),
        (HexCoord(-2, -1, 3), Planet("Starpoint", 1, 1, [PlanetTrait.HAZARDOUS])),
        (HexCoord(1, -1, 0), Planet("Tequ'ran", 2, 0, [PlanetTrait.HAZARDOUS])),
    ]
    
    for i, (coord, planet) in enumerate(sample_systems):
        system = System(id=20+i, planets=[planet])
        spaces.append(MapSpace(coord, MapSpaceType.SYSTEM, system))
    
    return TI4Map(spaces)

# Create the map
ti4_map = create_sample_map()
print(f"Created map with {len(ti4_map.spaces)} spaces")
print(f"Home systems: {len(ti4_map.get_home_spaces())}")
print(f"Planet systems: {len(ti4_map.get_system_spaces())}")

## 2. Define Evaluator

Create an evaluator similar to the "Joebrew" strategy from the JavaScript implementation.

In [ ]:
# Create Joebrew-style evaluator
joebrew_evaluator = Evaluator(
    name="Joebrew",
    PLANET_STRATEGY=PlanetEvalStrategy.GREATEST_PLUS_TECH,
    RESOURCES_MULTIPLIER=3.0,
    INFLUENCE_MULTIPLIER=2.0,
    TECH_MOD=5.0,
    MULTI_PLANET_MOD=1.0,
    MATCHING_PLANETS_MOD=1.0,
    MECATOL_REX_SYS_MOD=6.0,
    LEGENDARY_PLANET_SYS_MOD=6.0,
    DISTANCE_MULTIPLIER=[6, 6, 6, 4, 4, 2, 1, 0, 0, 0, 0]
)

print("✓ Evaluator created")

## 3. Analyze Initial Balance

Before optimization, let's see how balanced the map is.

In [ ]:
# Analyze initial balance
initial_analysis = analyze_balance(ti4_map, joebrew_evaluator)

print("=== Initial Balance Analysis ===")
print(f"Balance Gap: {initial_analysis['balance_gap']:.2f}")
print(f"Mean Value: {initial_analysis['mean']:.2f}")
print(f"Std Dev: {initial_analysis['std']:.2f}")
print(f"Jain's Fairness Index: {initial_analysis['fairness_index']:.3f}")
print(f"Min/Max: {initial_analysis['min']:.2f} / {initial_analysis['max']:.2f}")
print("\nHome Values:")
for i, value in initial_analysis['home_values']:
    print(f"  Player {i+1}: {value:.2f}")

## 4. Visualize Initial Map

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(16, 7))

# Plot map layout
plot_hex_map(ti4_map, ax=axes[0], title="Map Layout")

# Plot value heatmap
plot_value_heatmap(ti4_map, joebrew_evaluator, ax=axes[1], title="System Value Heatmap")

plt.tight_layout()
plt.show()

## 5. Run Balance Optimization

Now let's optimize the map balance using the iterative swapping algorithm.

In [ ]:
# Create a copy for optimization
optimized_map = ti4_map.copy()

# Run optimization
print("Running balance optimization...")
final_gap, history = improve_balance(
    optimized_map,
    joebrew_evaluator,
    iterations=200,
    random_seed=42
)

print(f"\n✓ Optimization complete")
print(f"Initial gap: {history[0][1]:.2f}")
print(f"Final gap: {final_gap:.2f}")
print(f"Improvement: {((history[0][1] - final_gap) / history[0][1] * 100):.1f}%")

## 6. Visualize Convergence

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))
plot_balance_convergence(history, ax=ax)
plt.show()

## 7. Compare Before and After

In [ ]:
# Analyze optimized map
final_analysis = analyze_balance(optimized_map, joebrew_evaluator)

print("=== Comparison ===")
print(f"Balance Gap:       {initial_analysis['balance_gap']:.2f} → {final_analysis['balance_gap']:.2f}")
print(f"Fairness Index:    {initial_analysis['fairness_index']:.3f} → {final_analysis['fairness_index']:.3f}")
print(f"Std Dev:           {initial_analysis['std']:.2f} → {final_analysis['std']:.2f}")

# Plot comparison
fig, ax = plt.subplots(figsize=(10, 6))

initial_values = [v for _, v in initial_analysis['home_values']]
final_values = [v for _, v in final_analysis['home_values']]

from ti4_analysis.visualization.map_viz import plot_balance_comparison
plot_balance_comparison(initial_values, final_values, ax=ax)
plt.show()

## 8. Advanced Spatial Statistics

Let's apply the spatial statistics from the research documentation.

In [ ]:
# Run comprehensive spatial analysis
spatial_results = comprehensive_spatial_analysis(optimized_map, joebrew_evaluator)

print("=== Spatial Statistics ===")
print(f"\nResource Clustering (Moran's I): {spatial_results['resource_clustering_morans_i']:.3f}")
print("  > 0: Resources are clustered")
print("  ≈ 0: Random distribution")
print("  < 0: Resources are dispersed")

print(f"\nHot Spots Detected: {spatial_results['num_hotspots']}")
print(f"Cold Spots Detected: {spatial_results['num_coldspots']}")

print(f"\nInequality Metrics:")
print(f"  Jain's Fairness Index: {spatial_results['jains_fairness_index']:.3f}")
print(f"  Gini Coefficient: {spatial_results['gini_coefficient']:.3f}")
print(f"  Coefficient of Variation: {spatial_results['inequality_metrics']['coefficient_of_variation']:.3f}")

print(f"\nGravity Model Accessibilities:")
for i, acc in enumerate(spatial_results['home_accessibilities']):
    print(f"  Player {i+1}: {acc:.2f}")

## 9. Identify Resource Hot Spots

In [ ]:
hotspots = identify_hotspots(optimized_map, joebrew_evaluator, significance_level=1.96)

if hotspots:
    print("=== Significant Hot/Cold Spots (95% confidence) ===")
    for coord, gi_star, spot_type in hotspots:
        print(f"{spot_type.upper():10s} at {coord.to_tuple()}: Gi* = {gi_star:.2f}")
else:
    print("No statistically significant hot or cold spots detected.")

## 10. Generate Comprehensive Report

In [ ]:
# Create full balance report
fig = create_balance_report(optimized_map, joebrew_evaluator, history, figsize=(16, 12))
plt.show()

## 11. Test Mathematical Properties

Let's verify some key mathematical properties of our algorithms.

In [ ]:
# Test 1: Triangle inequality for hex distance
print("=== Property Tests ===")
print("\n1. Triangle Inequality (hex distance)")
a = HexCoord(0, 0, 0)
b = HexCoord(2, -1, -1)
c = HexCoord(1, 1, -2)

d_ab = hex_distance(a, b)
d_bc = hex_distance(b, c)
d_ac = hex_distance(a, c)

print(f"d(a,b) = {d_ab}")
print(f"d(b,c) = {d_bc}")
print(f"d(a,c) = {d_ac}")
print(f"Triangle inequality: {d_ac} ≤ {d_ab} + {d_bc} = {d_ab + d_bc}")
print(f"✓ Verified: {d_ac <= d_ab + d_bc}")

# Test 2: Jain's Fairness Index range
print("\n2. Jain's Fairness Index Properties")
perfect_fair = np.array([10.0, 10.0, 10.0, 10.0])
print(f"Perfect fairness: {jains_fairness_index(perfect_fair):.3f} (should be 1.0)")

max_unfair = np.array([100.0, 0.0, 0.0, 0.0])
print(f"Maximum unfairness: {jains_fairness_index(max_unfair):.3f} (should be 0.25 for n=4)")

# Test 3: Optimization never increases gap
print("\n3. Optimization Monotonicity")
gaps = [gap for _, gap in history]
is_monotonic = all(gaps[i+1] <= gaps[i] for i in range(len(gaps)-1))
print(f"Gap never increases: {is_monotonic}")
if not is_monotonic:
    increases = sum(1 for i in range(len(gaps)-1) if gaps[i+1] > gaps[i])
    print(f"  (Found {increases} increases - this is expected due to greedy algorithm)")

## Summary

This notebook demonstrated:

1. **Balance Optimization**: Iterative swapping algorithm that reduces player advantage gaps
2. **Spatial Statistics**: Moran's I, Getis-Ord Gi*, and gravity models from research literature
3. **Fairness Metrics**: Jain's index, Gini coefficient, and other inequality measures
4. **Scientific Visualization**: Publication-quality plots using matplotlib and seaborn
5. **Mathematical Rigor**: Property-based testing and verification of key algorithms

### Next Steps

- Implement multi-objective optimization (Pareto fronts)
- Add betweenness centrality analysis
- Develop Voronoi-based territorial analysis
- Create statistical significance tests for balance differences
- Export optimized maps back to JavaScript format